In [63]:
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import re
import numpy as np
from pandas import ExcelWriter
import openpyxl

### Import survey responses and select relevant columns 

In [300]:
df = pd.read_csv(r'qualtrics_responses.csv')

In [301]:
print(df.columns.tolist())

['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage', 'Q3', 'Q4_1', 'Q4_2', 'Q4_3', 'Q5_1', 'Q5_2', 'Q6_1', 'Q7_1', 'Q8_1', 'Q9_1', 'Q10_1', 'Q11_1', 'Q12_1', 'Q12_2', 'Q13_1', 'Q14_1', 'Q15_1', 'Q15_2', 'Q16_1', 'Q16_2', 'Q17_1', 'Q18_1', 'Q18_2', 'Q19_1', 'Q20_1', 'Q20_2', 'Q20_3', 'Q21_1', 'Q22_1', 'Q23_1', 'Q23_2', 'Q24_1', 'Q24_2', 'Q25_1', 'Q26_1', 'Q27_1', 'Q28_1', 'Q29_1', 'Q30_1', 'Q31_1', 'Q32_1', 'Q33_1', 'Q34_1', 'Q34_2', 'Q35_1', 'Q36_1', 'Q37_1', 'Q37_2', 'Q37_3', 'Q37_4', 'Q38_1', 'Q38_2', 'Q38_3', 'Q39', 'Q46', 'Q40']


In [302]:
columns= ['Q4_1', 'Q4_2', 'Q4_3', 'Q5_1', 'Q5_2', 'Q6_1', 'Q7_1', 'Q8_1', 'Q9_1', 'Q10_1', 'Q11_1', 'Q12_1', 'Q12_2', 'Q13_1', 'Q14_1', 'Q15_1', 'Q15_2', 'Q16_1', 'Q16_2', 'Q17_1', 'Q18_1', 'Q18_2', 'Q19_1', 'Q20_1', 'Q20_2', 'Q20_3', 'Q21_1', 'Q22_1', 'Q23_1', 'Q23_2', 'Q24_1', 'Q24_2', 'Q25_1', 'Q26_1', 'Q27_1', 'Q28_1', 'Q29_1', 'Q30_1', 'Q31_1', 'Q32_1', 'Q33_1', 'Q34_1', 'Q34_2', 'Q35_1', 'Q36_1', 'Q37_1', 'Q37_2', 'Q37_3', 'Q37_4', 'Q38_1', 'Q38_2', 'Q38_3']

In [303]:
answers = df[columns]

In [304]:
pd.options.display.max_colwidth = 200
answers.head()

,Q4_1,Q4_2,Q4_3,Q5_1,Q5_2,Q6_1,Q7_1,Q8_1,Q9_1,Q10_1,...,Q34_2,Q35_1,Q36_1,Q37_1,Q37_2,Q37_3,Q37_4,Q38_1,Q38_2,Q38_3
0,"Si la regulación de manejo del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Innovación","Si la regulación de manejo del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Valorización del rastrojo","Si la regulación de manejo del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Sostenibilidad de la industria","Si la regulación de manejo del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Cantidad de mosca de establo","Si la regulación de manejo del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Uso de agroquimicos","Si la buena imagen de la industria aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Financiamiento","Si la colaboración/comunicación aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Innovación","Si la contaminación del suelo, aire y agua aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Salud/bienestar de la comu...","Si el costo de materiales basados en combustibles fósiles aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho).\n\n\n\nNota: produc...","Si el costo de materiales basados en combustibles fósiles aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Rentabilida...",...,"Si el tamaño de la empresa aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Rentabilidad de las empresas piñeras","Si la transparencia/apertura de la industria aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Colaboración/comunicación","Si el uso de agroquimicos aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Contaminación del suelo, aire y agua","Si la valorización del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Extracción del campo","Si la valorización del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Disponibilidad de la tierra","Si la valorización del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Rentabilidad de las empresas piñeras","Si la valorización del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Sostenibilidad de la industria","Si el valorización del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Fertilidad del suelo","Si el valorización del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Cantidad de mosca de establo","Si el valorización del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Uso de agroquimicos"
1,"{""ImportId"":""QID53_1""}","{""ImportId"":""QID53_2""}","{""ImportId"":""QID53_3""}","{""ImportId"":""QID85_1""}","{""ImportId""

In [305]:
#check if any expert didn't complete the survey
answers[answers.isna().any(axis=1)]

,Q4_1,Q4_2,Q4_3,Q5_1,Q5_2,Q6_1,Q7_1,Q8_1,Q9_1,Q10_1,...,Q34_2,Q35_1,Q36_1,Q37_1,Q37_2,Q37_3,Q37_4,Q38_1,Q38_2,Q38_3
6,1,5,5,4,Sin efecto,4,3,5,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,4,5,4,4,5,5,5,5,3,...,4,4,5,4,5,4,4,NaN,NaN,NaN
9,3,2,4,2,1,4,5,5,3,3,...,5,4,4,4,4,4,4,2,5,4


In [306]:
#answers.iloc[6]
#checking answers, we see that expert 7 didn't answer more than half, so we drop it. The other two only missed a few answers.
answers.drop([6], inplace = True)
answers=answers.reset_index(drop=True)
answers

,Q4_1,Q4_2,Q4_3,Q5_1,Q5_2,Q6_1,Q7_1,Q8_1,Q9_1,Q10_1,...,Q34_2,Q35_1,Q36_1,Q37_1,Q37_2,Q37_3,Q37_4,Q38_1,Q38_2,Q38_3
0,"Si la regulación de manejo del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Innovación","Si la regulación de manejo del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Valorización del rastrojo","Si la regulación de manejo del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Sostenibilidad de la industria","Si la regulación de manejo del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Cantidad de mosca de establo","Si la regulación de manejo del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Uso de agroquimicos","Si la buena imagen de la industria aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Financiamiento","Si la colaboración/comunicación aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Innovación","Si la contaminación del suelo, aire y agua aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Salud/bienestar de la comu...","Si el costo de materiales basados en combustibles fósiles aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho).\n\n\n\nNota: produc...","Si el costo de materiales basados en combustibles fósiles aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Rentabilida...",...,"Si el tamaño de la empresa aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Rentabilidad de las empresas piñeras","Si la transparencia/apertura de la industria aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Colaboración/comunicación","Si el uso de agroquimicos aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Contaminación del suelo, aire y agua","Si la valorización del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Extracción del campo","Si la valorización del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Disponibilidad de la tierra","Si la valorización del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Rentabilidad de las empresas piñeras","Si la valorización del rastrojo aumenta, ¿cuánto AUMENTAN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 aumenta poco y 5 aumenta mucho). - Sostenibilidad de la industria","Si el valorización del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Fertilidad del suelo","Si el valorización del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Cantidad de mosca de establo","Si el valorización del rastrojo aumenta, ¿cuánto DISMINUYEN los siguientes conceptos, en una escala de 1 a 5? (Siendo 1 disminuye poco y 5 disminuye mucho). - Uso de agroquimicos"
1,"{""ImportId"":""QID53_1""}","{""ImportId"":""QID53_2""}","{""ImportId"":""QID53_3""}","{""ImportId"":""QID85_1""}","{""ImportId""

### Create copy of dataframe, add + and - depending on connection between concepts 

In [307]:
answers1=answers.copy() 

In [308]:
for c in answers.columns:
    if answers[c].str.contains('AUMENTAN').any():
        answers1[c] = '+' + answers[c] 
    else:
        answers1[c] = '-' + answers[c]   
# another way
# for c in answers.columns:
#     answers1[c] = np.where(answers[c].str.contains('AUMENTAN').any(), answers[c] , answers[c] + 't')

In [309]:
# drop first rows
answers1=answers1.drop(answers1.index[[0,1]])
answers1.head()

,Q4_1,Q4_2,Q4_3,Q5_1,Q5_2,Q6_1,Q7_1,Q8_1,Q9_1,Q10_1,...,Q34_2,Q35_1,Q36_1,Q37_1,Q37_2,Q37_3,Q37_4,Q38_1,Q38_2,Q38_3
2,+3,+3,+3,-4,-2,+3,+4,-5,+Sin efecto,-2,...,+4,+4,+4,+5,+3,+4,+4,-2,-4,-2
3,+4,+5,+1,-4,-Sin efecto,+Sin efecto,+4,-5,+4,-2,...,+Sin efecto,+3,+4,+4,+3,+2,+3,-Sin efecto,-3,-Sin efecto
4,+5,+5,+3,-4,-4,+2,+4,-5,+2,-4,...,+3,+5,+4,+4,+5,+4,+4,-3,-5,-5
5,+4,+4,+5,-3,-3,+3,+4,-Sin efecto,+3,-3,...,+3,+3,+Sin efecto,+3,+3,+3,+3,-3,-3,-3
6,+5,+5,+5,-2,-2,+4,+4,-1,+4,-5,...,+5,+4,+5,+5,+5,+5,+5,-Sin efecto,-1,-5


### Extract concepts from questions and create lists: From and To

In [310]:
lFrom=[]
for column in answers.columns:
    fromC=re.search(r'Si (.*?) aumenta', answers[str(column)][0]).group(1)
    lFrom.append(fromC)

In [311]:
lTo=[]
for column in answers.columns:
    ToC=re.search(r'(?<= - ).*', answers[str(column)][0]).group(0)    
    lTo.append(ToC)

In [312]:
#transpose answers rows to columns
answersT=answers1.T
answersT.head()

,2,3,4,5,6,7,8,9
Q4_1,+3,+4,+5,+4,+5,+4,+3,+4
Q4_2,+3,+5,+5,+4,+5,+4,+2,+5
Q4_3,+3,+1,+3,+5,+5,+5,+4,+4
Q5_1,-4,-4,-4,-3,-2,-4,-2,-5
Q5_2,-2,-Sin efecto,-4,-3,-2,-4,-1,-5


###  Create list containing one dataframe with the answers for each expert

In [313]:
alldfs = []
for col in answersT.columns:
    df_current = answersT[[col]]
    alldfs.append(df_current)

In [314]:
# getting SettingWithCopyWarning, couldn't find solution so turning off for now. 
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [315]:
# add From To columns 
for i in range(len(alldfs)):
    alldfs[i].is_copy = False   # TURNS OFF SettingWithCopyWarning
    alldfs[i]['From'] = lFrom
    alldfs[i]['To'] = lTo
    alldfs[i].rename(columns={alldfs[i].columns[0]: "answers"}, inplace=True)

In [316]:
# fill in FCM matrix conditional on values of answers column
for i in range(len(alldfs)):
    alldfs[i].loc[alldfs[i]['answers'] == '-5', '-VH'] = 1
    alldfs[i].loc[alldfs[i]['answers'] == '-4', '-H'] = 1
    alldfs[i].loc[alldfs[i]['answers'] == '-3', '-M'] = 1
    alldfs[i].loc[alldfs[i]['answers'] == '-2', '-L'] = 1
    alldfs[i].loc[alldfs[i]['answers'] == '-1', '-VL'] = 1
    alldfs[i]['NA']=np.where(alldfs[i]['answers'].str.contains('Sin efecto'), 1 , 0)
    alldfs[i].loc[alldfs[i]['answers'] == '+1', '+VL'] = 1
    alldfs[i].loc[alldfs[i]['answers'] == '+2', '+L'] = 1
    alldfs[i].loc[alldfs[i]['answers'] == '+3', '+M'] = 1
    alldfs[i].loc[alldfs[i]['answers'] == '+4', '+H'] = 1
    alldfs[i].loc[alldfs[i]['answers'] == '+5', '+VH'] = 1
    alldfs[i].replace(np.nan,0, inplace=True)

In [317]:
alldfs[0].head()

,answers,From,To,-VH,-H,-M,-L,-VL,NA,+VL,+L,+M,+H,+VH
Q4_1,+3,la regulación de manejo del rastrojo,Innovación,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q4_2,+3,la regulación de manejo del rastrojo,Valorización del rastrojo,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q4_3,+3,la regulación de manejo del rastrojo,Sostenibilidad de la industria,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q5_1,-4,la regulación de manejo del rastrojo,Cantidad de mosca de establo,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
Q5_2,-2,la regulación de manejo del rastrojo,Uso de agroquimicos,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [318]:
#drop answer column
for i in range(len(alldfs)):
    alldfs[i]=alldfs[i].drop(columns='answers', axis=1)

In [319]:
# we want concepts to have the same name in From and To column for the sake of the FCM matrix
for x in lTo:
    for i in range(len(alldfs)):
        alldfs[i].loc[alldfs[i]['From'].str.contains(x, case=False), "From"]= x

In [320]:
for i in range(len(alldfs)):
        alldfs[i]['From'] = alldfs[i]['From'].str.replace(r'^las ', '',1, regex=True)
        alldfs[i]['From'] = alldfs[i]['From'].str.replace(r'^la ', '',1, regex=True)
        alldfs[i]['From'] = alldfs[i]['From'].str.replace(r'^el ', '',1, regex=True)

In [321]:
for i in range(len(alldfs)):
    alldfs[i]['From'] = alldfs[i]['From'].str.replace(r'(\w+)', lambda x: x.group().capitalize(), n=1, regex=True)

In [322]:
alldfs[0]

,From,To,-VH,-H,-M,-L,-VL,NA,+VL,+L,+M,+H,+VH
Q4_1,Regulación de manejo del rastrojo,Innovación,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q4_2,Regulación de manejo del rastrojo,Valorización del rastrojo,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q4_3,Regulación de manejo del rastrojo,Sostenibilidad de la industria,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q5_1,Regulación de manejo del rastrojo,Cantidad de mosca de establo,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
Q5_2,Regulación de manejo del rastrojo,Uso de agroquimicos,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0
Q6_1,Buena imagen de la industria,Financiamiento,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q7_1,Colaboración/comunicación,Innovación,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0
Q8_1,"Contaminación del suelo, aire y agua",Salud/bienestar de la comunidad,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
Q9_1,Costo de materiales basados en combustibles fósiles,Demanda de productos PAL,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0
Q10_1,Costo de materiales basados en combustibles fósiles,Rentabilidad de las empresas piñeras,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [325]:
pd.unique(sorted(alldfs[0][['From', 'To']].values.ravel('K')))

array(['Buena imagen de la industria', 'Cantidad de mosca de establo',
       'Colaboración/comunicación',
       'Contaminación del suelo, aire y agua',
       'Costo de materiales basados en combustibles fósiles',
       'Costumbres de la industria', 'Demanda de productos PAL',
       'Desnivel del terreno', 'Disponibilidad de la tierra', 'Empleo',
       'Extracción del campo', 'Fertilidad del suelo', 'Financiamiento',
       'Inestabilidad internacional ', 'Innovación',
       'Investigación académica', 'Lluvia', 'Participación del gobierno',
       'Preferencia de los consumidores por productos sostenibles',
       'Productividad de la producción de piña',
       'Productividad de los ganaderos', 'Productividad laboral',
       'Regulaciones y estándares de importación (de destino)',
       'Regulación de manejo del rastrojo',
       'Rentabilidad de las empresas piñeras', 'Riesgo empresarial',
       'Salud/bienestar de la comunidad',
       'Sostenibilidad de la industria', 'Tam

In [156]:
#concepts=[f'c{i}' for i in range(1, 34)]
#print(concepts)

In [155]:
#alldfs[1].replace(From,concepts)

### Create dictionary wirth expert number as key

In [326]:
experts=[f'expert{i}' for i in range(1, 10)]
print(experts)

['expert1', 'expert2', 'expert3', 'expert4', 'expert5', 'expert6', 'expert7', 'expert8', 'expert9']


In [327]:
experDic = dict(zip(experts, alldfs))

In [328]:
experDic.keys()

dict_keys(['expert1', 'expert2', 'expert3', 'expert4', 'expert5', 'expert6', 'expert7', 'expert8'])

### Now we have a dictionary containing one dataframe for each expert. This will be imported in the FCMpy script

In [329]:
experDic['expert1'].head()

,From,To,-VH,-H,-M,-L,-VL,NA,+VL,+L,+M,+H,+VH
Q4_1,Regulación de manejo del rastrojo,Innovación,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q4_2,Regulación de manejo del rastrojo,Valorización del rastrojo,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q4_3,Regulación de manejo del rastrojo,Sostenibilidad de la industria,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0
Q5_1,Regulación de manejo del rastrojo,Cantidad de mosca de establo,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
Q5_2,Regulación de manejo del rastrojo,Uso de agroquimicos,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0


### Save to excel, each dataframe is one sheet 

In [330]:
def save_xls(dict_df, path):
    """
    Save a dictionary of dataframes to an excel file, 
    with each dataframe as a separate sheet
    """

    writer = ExcelWriter(path)
    for key in dict_df.keys():
        dict_df[key].to_excel(writer, sheet_name=key, index=False)

    writer.save()
    writer.close()

In [331]:
save_xls(experDic, "expertsR.xlsx")